In [1]:
import math
import numpy
from pyevtk.hl import pointsToVTK
import numba

In [2]:
@numba.njit(parallel=True, fastmath=True)
def generate_point_cloud(m_n_points, m_range):
    return m_range*numpy.random.rand(m_n_points,3);

In [3]:
def write_to_file(m_n_points, m_points, m_iter, m_range, m_t):
    r = check_if_below(m_n_points, m_points, m_iter, m_range);
    x, y, z = numpy.array(m_points[:,0]), numpy.array(m_points[:,1]), numpy.array(m_points[:,2]);    
    pointsToVTK("./results/points_"+str(m_t), x, y, z, data={"thres": r});

In [4]:
@numba.njit(parallel=True, fastmath=True)
def check_if_below(m_n_points, m_points, m_iter, m_range):
    m_r = numpy.zeros(m_n_points);
    m_color = numpy.zeros(m_n_points);
    for i in range(m_n_points):
        x_iter = 0.022;
        y_iter = 0.03;
        z_iter = 0.014;
        radius = 0.;
        for itr in range(20):            
            if radius < 1.:
                radius = math.sqrt(x_iter**2+y_iter**2+z_iter**2);
                theta = math.atan2(math.sqrt(x_iter**2+y_iter**2), z_iter);
                phi = math.atan2(y_iter, x_iter);
                x_iter = (radius**m_iter)*math.sin(m_iter*theta)*math.cos(m_iter*phi)+m_points[i][0];
                y_iter = (radius**m_iter)*math.sin(m_iter*theta)*math.sin(m_iter*phi)+m_points[i][1];
                z_iter = (radius**m_iter)*math.cos(m_iter*theta)+m_points[i][2];                
                m_r[i] = radius;
            else: 
                m_r[i] = -1;
    return m_r;

In [5]:
n_points = 1.E6;
iteration_max = 20.;
L = 1.1;
point_cloud = generate_point_cloud(n_points, L);
t_max = 0;
for t in range(t_max):
    power_t = t/t_max*iteration_max;
    write_to_file(n_points, point_cloud, power_t, L, t);

RuntimeError: Failed in nopython mode pipeline (step: nopython mode backend)
LLVM IR parsing error
<string>:342:18: error: invalid cast opcode for cast from 'double' to 'i64'
  %".263" = sext double %".259" to i64
                 ^
